In [1]:
import random
import json
import pickle
import numpy as np
import requests

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.load(open('intents.json'))

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

#Maakt de zinnen schoon, checkt alle woorden en brengt het terug naar de stam van het woord.
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

#In deze functie wordt de zin omgezet naar losse woorden en 1en en 0en.
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] *len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] =1
    return np.array(bag)

#In deze functie wordt een voorspelling gedaan van de zin in de bag of words. Hieruit wordt er een result voorspelt. 
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    result.sort(key=lambda x: x[1])
    return_list = []
    for r in result:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

#In deze functie wordt er een antwoord opgehaald die de chatbot zal gaan zeggen op basis van input van de gebruiker 
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
   
#In deze functie wordt een API-call gedaan om een random grap te vertellen aan de user als de chatbot een pattern herkent gerelateerd aan grappen.
def temp(message, intents):    
    for intent in intents['intents']:
        if message in intent['patterns']:
            if intent['tag'] == 'jokes':
                data = requests.get("https://official-joke-api.appspot.com/random_joke")
                data = json.loads(data.content)
                data = data['setup'] +" "+ data["punchline"]
                return data
            

print("Go! Bot is running!")
print("If you want to stop talking just say 'stop' to me!")

while True:
    message = input("").lower()
    if message == "stop":
        break

    resp2 = temp(message, intents)
    print("You: " + message)
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Chatman: " + res)
    if resp2 != None:
        print(resp2)
    else:
        continue

Go! Bot is running!
If you want to stop talking just say 'stop' to me!
You: jokes
1/1 [==============================] - 0s 316ms/step
Chatman: I know a fun Joke! Wait a second! 
You: joke
1/1 [==============================] - 0s 66ms/step
Chatman: I know a fun Joke! Wait a second! 
Want to hear a chimney joke? Got stacks of em! First one's on the house
You: 
1/1 [==============================] - 0s 45ms/step
Chatman: Goodbye, have a good one!
